In [1]:
import pandas as pd
import copy
from datetime import datetime as dt

### Загрузка сырых данных

In [45]:
kb = pd.read_csv('./keyboard_A_non_insider_1.csv', sep='|', header=None)
kb.info()
print(kb.head(200).to_string())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
0    172 non-null object
1    172 non-null object
2    172 non-null object
dtypes: object(3)
memory usage: 4.2+ KB
                           0        1              2
0    2019-12-04 04:25:14,553  release      Key.enter
1    2019-12-04 04:25:25,621    press        Key.esc
2    2019-12-04 04:25:25,699  release        Key.esc
3    2019-12-04 04:25:26,632    press      Key.space
4    2019-12-04 04:25:26,728  release      Key.space
5    2019-12-04 04:25:27,562    press  Key.backspace
6    2019-12-04 04:25:27,651  release  Key.backspace
7    2019-12-04 04:25:27,808    press  Key.backspace
8    2019-12-04 04:25:27,884  release  Key.backspace
9    2019-12-04 04:25:28,127    press  Key.backspace
10   2019-12-04 04:25:28,192  release  Key.backspace
11   2019-12-04 04:25:29,397    press        Key.cmd
12   2019-12-04 04:25:29,493  release        Key.cmd
13   2019-12-04 04:25:30,449    press  

In [7]:
ms = pd.read_csv('./mouse_A_non_insider_1.csv', sep='|', header=None)
ms.info()
ms.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
0    36 non-null object
1    36 non-null object
2    36 non-null object
dtypes: object(3)
memory usage: 992.0+ bytes


,0,1,2
0,"2019-12-04 04:25:16,140",press,Button.left
1,"2019-12-04 04:25:16,235",release,Button.left
2,"2019-12-04 04:25:16,670",press,Button.left
3,"2019-12-04 04:25:16,788",release,Button.left
4,"2019-12-04 04:25:17,751",press,Button.right
5,"2019-12-04 04:25:17,757",release,Button.right
6,"2019-12-04 04:25:18,909",press,Button.left
7,"2019-12-04 04:25:18,973",release,Button.left
8,"2019-12-04 04:25:19,984",press,Button.left
9,"2019-12-04 04:25:20,087",release,Button.left


### Списки выделяемых признаков

In [8]:
# кнопки мыши
mouse_buttons = [
    "Button.left",
    "Button.right",
]

# специальные клавиши
special_keys = [
    "Key.esc",
    "Key.tab",
    "Key.caps_lock",
    "Key.shift",
    "Key.ctrl",
    "Key.alt",
    "Key.cmd",
    "Key.space",
    "Key.enter",
    "Key.backspace",
]

# диграфы и триграфы - сочетания из 2-х и 3-х букв
eng_di = ["th", "he", "in", "er", "an", "re", "es", "on", "st", "nt", "en", "at", "ed", "nd", "to", "or", "ea"]
eng_tri = ["the", "and", "ing", "ent", "ion", "her", "for", "tha", "nth", "int", "ere", "tio", "ter", "est", "ers", "ati", "hat"]
ru_di = ["ст","ен","ов","но","ни","на","ра","ко","то","ро","ан","ос","по","го","ер","од", "ре"]
ru_tri = ["ени","ост","ого","ств","ско","ста","ани","про","ест","тор","льн","ова","ния","ние","при","енн","год"]

# признаки для специальных признаков и кнопок мыши
spec_features = {
    "dwell": [0, 0], # длительность нажатия
    "interval" : [0, 0], # промежуток между отпусканием текущей и нажатием следующей
    "flight": [0,0] # промежуток между нажатием текущей и нажатием следующей
}

# признаки для диграфов
di_features = {
    "dwell_first": [0,0], # длительность нажатия первой буквы
    "dwell_second": [0,0], # длительность нажатия второй буквы
    "interval": [0,0], # промежуток между отпусканием первой и нажатием второй буквы
    "flight": [0,0], # промежуток между нажатием первой и нажатием второй буквы
    "up_to_up": [0,0], # промежуток между отпусканием первой и отпусканием второй
    "latency": [0,0], # промежуток между нажатием первой и отпусканием второй
}

# признаки для триграфов
tri_features = {
    "dwell_first": [0,0], # длительность нажатия первой буквы
    "dwell_second": [0,0], # длительность нажатия второй буквы
    "dwell_third": [0,0], # длительность нажатия третьей буквы
    "interval_first": [0,0], # промежуток между отпусканием первой и нажатием второй буквы
    "interval_seconds": [0,0], # промежуток между отпусканием второй и нажатием третьей буквы
    "flight_first": [0,0], # промежуток между нажатием первой и нажатием второй буквы
    "flight_second": [0,0], # промежуток между нажатием второй и нажатием третьей буквы    
    "up_to_up_first": [0,0], # промежуток между отпусканием первой и отпусканием второй
    "up_to_up_second": [0,0], # промежуток между отпусканием второй и отпусканием третьей
    "latency": [0,0], # промежуток между нажатием первой и отпусканием третьей
}

### Выделение признаков

In [46]:
# здесь будут храниться все вышеприведённые признаки
features = {}

class FeatureProcessor:
    # вычисление признаков для кнопок мыши
    def mouse_and_special_keys(self, df, events):
        for event in events:
            if event not in features: # в словаре признаков пока не выделена память под эту кнопку
                        features[event] = copy.deepcopy(spec_features) # выделяем память
            for i, row in df.iterrows():
                if row[1] == "press" and row[2] == event:
                    press = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f') # фиксируем время нажатия кнопки
                    # время отпускания нажатой кнопки 
                    # по умолчанию задаётся значением press, 
                    # так как release для последнего события в датасете может отсутствовать
                    release = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                    # время нажатия кнопки после отпускания текущей
                    # по умолчанию задаётся значением press, 
                    # так как press следующей кнопки для последнего события в датасете может отсутствовать
                    press_next = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                    j = 1
                    while True: # ищем время отпускания кнопки
                        try: # пробуем обратиться по индексу i + j
                            next_row = df.iloc[i + j]
                            if next_row[1] == "release" and next_row[2] == event:
                                # фиксируем время отпускания текущей кнопки
                                release = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                try: # пробуем обратиться по индексу i +j + 1
                                    # фиксируем время нажатия следующей кнопки
                                    press_next = dt.strptime(df.iloc[i + j + 1][0], '%Y-%m-%d %H:%M:%S,%f')
                                except IndexError: # в случае отсутствия такого индекса, т.е. в случае выхода за границу датафрейма
                                    # фиксируем время нажатия следующей кнопки, как время отпускания текущей
                                    press_next = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                finally:
                                    # выходим из вспомогательного цикла в любом случае, так как release был найден
                                    break
                            else:
                                j+=1 # переход к следующей строке
                        except IndexError:
                            # выходим из вспомогательного цикла
                            break
                    # считаем признаки
                    # dwell
                    features[event]["dwell"][0] += (release - press).microseconds // 1000 # прибавляем длительность
                    features[event]["dwell"][1] += 1 # увеличиваем количество обработанных кнопок event
                    # interval
                    features[event]["interval"][0] += (press_next - release).microseconds // 1000 # прибавляем длительность
                    features[event]["interval"][1] += 1 # увеличиваем количество обработанных кнопок event
                    # flight
                    features[event]["flight"][0] += (press_next - press).microseconds // 1000 # прибавляем длительность
                    features[event]["flight"][1] += 1 # увеличиваем количество обработанных кнопок event
                
    def digraph_features(self, df, events):
        for event in events:
            if event not in features: # в словаре признаков пока не выделена память под эту кнопку
                        features[event] = copy.deepcopy(di_features) # выделяем память
            k = 0
            first_press = ""
            first_release = ""
            for i, row in df.iterrows():
                if row[1] == "press":
                    if row[2][1:-1] == event[k]:
                        press = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f') # фиксируем время нажатия кнопки
                        # время отпускания нажатой кнопки 
                        # по умолчанию задаётся значением press, 
                        # так как release для последнего события в датасете может отсутствовать
                        release = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                        # время нажатия кнопки после отпускания текущей
                        # по умолчанию задаётся значением press, 
                        # так как press следующей кнопки для последнего события в датасете может отсутствовать
                        press_next = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                        j = 1
                        while True: # ищем время отпускания кнопки
                            try: # пробуем обратиться по индексу i + j
                                next_row = df.iloc[i + j]
                                if next_row[1] == "release" and next_row[2][1:-1] == event[k]:
                                    # фиксируем время отпускания текущей кнопки
                                    release = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                    try: # пробуем обратиться по индексу i +j + 1
                                        # фиксируем время нажатия следующей кнопки
                                        press_next = dt.strptime(df.iloc[i + j + 1][0], '%Y-%m-%d %H:%M:%S,%f')
                                    except IndexError: # в случае отсутствия такого индекса, т.е. в случае выхода за границу датафрейма
                                        # фиксируем время нажатия следующей кнопки, как время отпускания текущей
                                        press_next = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                    finally:
                                        # выходим из вспомогательного цикла в любом случае, так как release был найден
                                        break
                                else:
                                    j+=1 # переход к следующей строке
                            except IndexError:
                                # выходим из вспомогательного цикла
                                break
                        if k == 0:
                            # сохраняем момент нажатия и отпускания первой клавиши
                            first_press = press
                            first_release = release
                        else:
                            print(event, first_press, first_release, press, release, press_next)
                            # считаем признаки
                            # dwell_first
                            features[event]["dwell_first"][0] += (first_release - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["dwell_first"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # interval
                            features[event]["interval"][0] += (press - first_release).microseconds // 1000 # прибавляем длительность
                            features[event]["interval"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # flight
                            features[event]["flight"][0] += (press - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["flight"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # dwell_second
                            features[event]["dwell_second"][0] += (release - press).microseconds // 1000 # прибавляем длительность
                            features[event]["dwell_second"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # up_to_up
                            features[event]["up_to_up"][0] += (release - first_release).microseconds // 1000 # прибавляем длительность
                            features[event]["up_to_up"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # latency
                            features[event]["latency"][0] += (release - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["latency"][1] += 1 # увеличиваем количество обработанных кнопок event 
                            k = 0
                            first_press = ""
                            first_release = ""
                        k+=1
                    else:
                        k = 0
                        first_press = ""
                        first_release = ""

In [47]:
fp = FeatureProcessor()
fp.mouse_and_special_keys(ms, mouse_buttons)
fp.mouse_and_special_keys(kb, special_keys)
fp.digraph_features(kb, ru_di)
for k,v in features.items():
    print(k,v)

на 2019-12-08 02:52:51.140000 2019-12-08 02:52:51.224000 2019-12-08 02:52:51.289000 2019-12-08 02:52:51.361000 2019-12-08 02:52:51.406000
то 2019-12-08 02:52:53.125000 2019-12-08 02:52:53.181000 2019-12-08 02:52:53.260000 2019-12-08 02:52:53.324000 2019-12-08 02:52:53.370000
по 2019-12-08 02:52:52.863000 2019-12-08 02:52:52.943000 2019-12-08 02:52:52.970000 2019-12-08 02:52:53.010000 2019-12-08 02:52:53.125000
ер 2019-12-08 02:52:53.930000 2019-12-08 02:52:54.026000 2019-12-08 02:52:54.040000 2019-12-08 02:52:54.124000 2019-12-08 02:52:55.089000
Button.left {'dwell': [1821, 15], 'interval': [6481, 15], 'flight': [7302, 15]}
Button.right {'dwell': [19, 3], 'interval': [1309, 3], 'flight': [1328, 3]}
Key.esc {'dwell': [78, 1], 'interval': [933, 1], 'flight': [11, 1]}
Key.tab {'dwell': [0, 0], 'interval': [0, 0], 'flight': [0, 0]}
Key.caps_lock {'dwell': [0, 0], 'interval': [0, 0], 'flight': [0, 0]}
Key.shift {'dwell': [2044, 7], 'interval': [1585, 7], 'flight': [1629, 7]}
Key.ctrl {'dwel